In [ ]:
pip install flexvalue --upgrade

In [ ]:
!flexvalue download-avoided-costs-data-db --year 2020

In [ ]:
import os
try:
    # only works from colab
    from google.colab import files
except:
    if not os.environ.get('DATABASE_LOCATION_ORIG'):
        os.environ["DATABASE_LOCATION_ORIG"] = os.environ.get('DATABASE_LOCATION', '.')
    os.environ["DATABASE_LOCATION"] = '../' + os.environ['DATABASE_LOCATION_ORIG'] 
import pandas as pd

pd.set_option('display.max_rows', None)

In [ ]:
from flexvalue.cet_flexvalue_compare import CET_Scan
from pathlib import Path

acc_version = '2020'
directory = 'cet_scan'
scan_name = 'Test_Run'
cet_scan = CET_Scan(
    directory=directory,
    scan_name=scan_name,
    program_year="2021",
    acc_version=acc_version,
    program_admin="PGE",
    climate_zone=["12", "3A", "4"],
    mwh=[5000, 10, 2],
    therms=[200, 3, 114],
    units=[1, 1, 10],
    ntg=[0.95, 0.4, 0.9],
    eul=[7, 1, 1],
    sector=["NonRes", "Res", "Res"],
    deer_load_shape=[
        "DEER:HVAC_Chillers",
        "DEER:HVAC_Chillers",
        "DEER:Indoor_CFL_Ltg",
    ],
    gas_sector=["Residential", "Residential", "Residential"],
    gas_savings_profile=["Annual", "Winter Only", "Summer Only"],
    admin_cost=[5000, 165, 299],
    measure_cost=[220000, 310, 4044],
    incentive=[200000, 66, 800],
)

user_inputs = cet_scan.generate_cet_input_file()
print("Input and output files will be made available for download in subsequent cells.")
user_inputs

In [ ]:
# This step downloads the CET zip file that you can then upload to CEDARS
cet_zip_filepath = os.path.join(cet_scan.cet_path, f'{scan_name}.zip')
files.download(cet_zip_filepath)
print("Make sure to set the CET Run Parameters with the following:\n- First year: 2021\n- Avoided Costs: 2021\n- Market Effects: 0%")
print('https://cedars.sound-data.com/accounts/login/?next=/cet_ui/')

In [ ]:
from flexvalue import FlexValueRun
flx_run = FlexValueRun(database_year=acc_version)
outputs_table, outputs_table_totals, electric_benefits, gas_benefits = flx_run.get_results(user_inputs=user_inputs)

In [ ]:
display(outputs_table)

In [ ]:
# Run this cell if you want to download the FLEXvalue output.
outputs_filename = os.path.join(cet_scan.flexvalue_path, 'flexvalue_outputs.csv')
outputs_table.to_csv(outputs_filename)
files.download(outputs_filename)

In [ ]:
# Upload the cet output file
cet_output_dict = files.upload()
if len(cet_output_dict) == 1:
    cet_output_filepath = list(cet_output_dict.keys())[0]
else:
    print("Must select exactly one file")

In [ ]:
cet_cleaned_results = cet_scan.parse_cet_output(cet_output_filepath)
compared_df = cet_scan.compare_cet_to_flexvalue(cet_cleaned_results, outputs_table)
compared_df

In [ ]:
from flexvalue.plots import plot_results
plot_results(outputs_table_totals, electric_benefits, gas_benefits)